In [1]:
import argparse
import logging
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
from utils.dice_score import dice_loss
from evaluate import evaluate
from unet_model import UNet
from wholeslidedata.iterators import create_batch_iterator
import numpy as np
from matplotlib import pyplot as plt
from plot_utils import init_plot, plot_batch, show_plot
from shapely.prepared import prep
#import albumentations as A

In [2]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
dir_checkpoint = Path('./checkpoints/')
from utils.dice_score import dice_loss
from evaluate import evaluate
from unet_model import UNet

cuda


In [4]:
def dice_loss(output, target):

    eps = 0.0001

    intersection = output * target
    numerator = 2 * intersection.sum(0).sum(1).sum(1)
    denominator = output + target
    denominator = denominator.sum(0).sum(1).sum(1) + eps
    loss_per_channel = (1 - (numerator / denominator))

    return loss_per_channel.sum() / output.size(1)

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.reshape(-1)
        targets = targets.reshape(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

In [5]:
# number of batches
batches = 64
# batch iterator config
user_config = 'user_config.yml'
# number of cpus used to extract patches on multiple cores
cpus = 1

# create iterators for the dataset
if torch.cuda.is_available():
    try:
        training_iterator = create_batch_iterator(user_config=user_config, mode='training', cpus=1, number_of_batches=batches)
        test_iterator = create_batch_iterator(user_config=user_config, mode='validation', cpus=1, number_of_batches=batches)    
    except:
        print("Exception!!!")
        sys.exit()

In [6]:
net = UNet(n_channels=3, n_classes=2, bilinear=True)
net.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr= 1e-4, weight_decay=0.0001)

In [7]:
train_losses = []
test_losses = []

textfile = open("train_dice_loss.txt", "w")
textfile.close()
textfile = open("test_dice_loss.txt", "w")
textfile.close()

In [8]:
for epoch in range(30):
    running_train_loss = 0.0
    running_test_loss = 0.0
    with tqdm(training_iterator, unit="batch") as tepoch:
        for i, data in enumerate(tepoch, 0):
            tepoch.set_description(f"Epoch {epoch}")
            
            # normalize the data
            images, true_masks = data[0], data[1]
            info = data[2]
            
            # convert images to tensors
            images = images.astype(np.float64)
            true_masks = true_masks.astype(np.float64)
            images = torch.Tensor(images)
            true_masks = torch.Tensor(true_masks)
            images = torch.permute(images,(0,3,1,2))
            assert images.shape[1] == net.n_channels, \
                f'Network has been defined with {net.n_channels} input channels, ' \
                f'but loaded images have {images.shape[1]} channels. Please check that ' \
                'the images are loaded correctly.'
            images = images.to(device=device, dtype=torch.float32)/255
            true_masks = true_masks.to(device=device, dtype=torch.long)
            
            pred_masks = net(images)
            loss = criterion(pred_masks, true_masks) \
                               + dice_loss(F.softmax(pred_masks, dim=1).float(),
                                           F.one_hot(true_masks, net.n_classes).permute(0, 3, 1, 2).float())
            
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #detaching
            loss_train = loss.detach().cpu().item()
            
            #tracking_loss
            running_train_loss += loss_train
            tepoch.set_postfix(loss=loss_train)
            
            #cleanup
            del images
            del true_masks
            del loss
            del pred_masks
            
            torch.cuda.empty_cache()
    print("Training Done")
    with torch.no_grad():
        for i, data in enumerate(test_iterator, 0):
            images, true_masks = data[0], data[1]
            info = data[2] 
            
            # convert images to tensors
            images = images.astype(np.float64)
            true_masks = true_masks.astype(np.float64)
            images = torch.Tensor(images)
            true_masks = torch.Tensor(true_masks)
            images = torch.permute(images,(0,3,1,2))
            assert images.shape[1] == net.n_channels, \
                f'Network has been defined with {net.n_channels} input channels, ' \
                f'but loaded images have {images.shape[1]} channels. Please check that ' \
                'the images are loaded correctly.'
            images = images.to(device=device, dtype=torch.float32)/255
            true_masks = true_masks.to(device=device, dtype=torch.long)
            
            pred_masks = net(images)
            loss = criterion(pred_masks, true_masks) \
                               + dice_loss(F.softmax(pred_masks, dim=1).float(),
                                           F.one_hot(true_masks, net.n_classes).permute(0, 3, 1, 2).float())
            #detaching
            loss_test = loss.detach().cpu().item()
            
            #running_loss
            running_test_loss += loss_test
            
            #cleanup
            del images
            del true_masks
            del loss
            del pred_masks
            
            torch.cuda.empty_cache()
    print("Validation Done")
    textfile = open("train_dice_loss.txt", "a")
    textfile.write(str(running_train_loss/len(training_iterator)) + "\n")
    textfile.close()

    train_losses.append(running_train_loss/len(training_iterator))

    textfile = open("test_dice_loss.txt", "a")
    textfile.write(str(running_test_loss/len(test_iterator)) + "\n")
    textfile.close()

    test_losses.append(running_test_loss/len(test_iterator))
    print(train_losses[-1])
    print(test_losses[-1])
    # LR scheduler step
    #scheduler.step(loss_test)
    # save checkpoint weights every 20 epochs 
    if ((epoch + 1) % 2 == 0):
        torch.save(net.state_dict(), './checkpoints/weights_' + str(epoch+1) + '.pth')

Epoch 0: 100%|██████████| 64/64 [00:44<00:00,  1.44batch/s, loss=0.958]


Training Done


Epoch 1:   0%|          | 0/64 [00:00<?, ?batch/s]

Validation Done
1.108336791396141
1.036374220624566


Epoch 1: 100%|██████████| 64/64 [00:44<00:00,  1.44batch/s, loss=0.846]


...

Training Done


Epoch 29:   0%|          | 0/64 [00:00<?, ?batch/s]

Validation Done
0.5129474471323192
1.9471442755311728


Epoch 29: 100%|██████████| 64/64 [00:44<00:00,  1.44batch/s, loss=0.543]


Training Done
Validation Done
0.5280507509596646
1.9370951037853956
